<a href="https://colab.research.google.com/github/gowtham1997/indicTrans/blob/main/indictrans_fairseq_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# create a seperate folder to store everything
!mkdir testing
%cd testing

/content/testing


In [ ]:
# clone the repo for running evaluation
!git clone https://github.com/AI4Bharat/indicTrans.git
%cd IndicTrans_evaluation
# clone requirements repositories
!git clone https://github.com/anoopkunchukuttan/indic_nlp_library.git
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git
!git clone https://github.com/rsennrich/subword-nmt.git
%cd ..

Cloning into 'IndicTrans_evaluation'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 21 (delta 3), reused 21 (delta 3), pack-reused 0
Unpacking objects: 100% (21/21), done.
/content/testing/IndicTrans_evaluation
Cloning into 'indic_nlp_library'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 1271 (delta 50), reused 54 (delta 25), pack-reused 1178
Receiving objects: 100% (1271/1271), 9.56 MiB | 12.32 MiB/s, done.
Resolving deltas: 100% (654/654), done.
Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 133 (delta 0), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (133/133), 149.77 MiB | 25.53 MiB/s, done.
Resolving deltas: 100% (51/51), done.
C

In [ ]:
# Install the necessary libraries
!pip install sacremoses pandas mock sacrebleu tensorboardX pyarrow indic-nlp-library
# Install fairseq from source
!git clone https://github.com/pytorch/fairseq.git
%cd fairseq
# !git checkout da9eaba12d82b9bfc1442f0e2c6fc1b895f4d35d
!pip install --editable ./
%cd ..

     |████████████████████████████████| 870kB 12.6MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 122kB 29.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=720de8c556f500516ba275cdf1c76f250a77e2b2461bd86bbb1e3458a1f1a24e
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
Cloning into 'fairseq'...
remote: Enumerating objects: 27579, done.
remote: Total 27579 (delta 0), reused 0 (delta 0), pack-reused 27579
Receiving objects: 100% (27579/27579), 11.43 MiB | 7.93 MiB/s, done.
Resolving deltas: 100% (20815/20815), done.
/content/testing/fairseq
Obtaining file:///content/testing/fairseq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 133kB 10.0

In [ ]:
# download the indictrans model
# in this notebook, we will be testing the indictrans-en-indic model
# To test the indic-en model, pls use this link ->  https://storage.googleapis.com/samanantar-public/models/indictrans-indic-en.zip

!wget https://storage.googleapis.com/samanantar-public/models/indictrans-en-indic.zip
!unzip indictrans-en-indic.zip
%cd IndicTrans_evaluation/

--2021-04-12 14:13:49--  https://storage.googleapis.com/samanantar-public/models/indictrans-en-indic.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.127.128, 172.217.218.128, 108.177.126.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.127.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4382161679 (4.1G) [application/zip]
Saving to: ‘indictrans-en-indic.zip’

indictrans-en-indic 100%[===================>]   4.08G  69.0MB/s    in 53s     

2021-04-12 14:14:43 (78.8 MB/s) - ‘indictrans-en-indic.zip’ saved [4382161679/4382161679]

Archive:  indictrans-en-indic.zip
   creating: indictrans-en-indic/
   creating: indictrans-en-indic/final_bin/
  inflating: indictrans-en-indic/final_bin/dict.SRC.txt  
  inflating: indictrans-en-indic/final_bin/dict.TGT.txt  
   creating: indictrans-en-indic/model/
  inflating: indictrans-en-indic/model/checkpoint_best.pt  
   creating: indictrans-en-indic/vocab/
  inflating: 

In [ ]:
# creating a text file and adding en sentences we can use for testing the model
!touch en_sentences.txt
!echo 'This bicycle is too small for you!!' >> en_sentences.txt
!echo ' I am running late. Meet you directly at the airport' >> en_sentences.txt
!echo 'In 2006, Time magazine stated that the open-door policy of allowing anyone to edit had made Wikipedia the "biggest and perhaps the best encyclopedia in the world", and a testament to the vision of Jimmy Walest' >> en_sentences.txt

In [ ]:
# joint_translate takes src_file, output_fname, src_lang, tgt_lang, model_folder as inputs
# src_file -> input text file to be translated
# output_fname -> name of the output file (will get created) containing the model predictions
# src_lang -> source lang code of the input text ( in this case we are using en-indic model and hence src_lang would be 'en')
# tgt_lang -> target lang code of the input text ( tgt lang for en-indic model would be any of the 11 indic langs we trained on:
#              as, bn, hi, gu, kn, ml, mr, or, pa, ta, te)
# supported languages are:
#              as - assamese, bn - bengali, gu - gujarathi, hi - hindi, kn - kannada, 
#              ml - malayalam, mr - marathi, or - oriya, pa - punjabi, ta - tamil, te - telugu

# model_dir -> the directory containing the model and the vocab files

# Note: if the translation is taking a lot of time, please tune the buffer_size and batch_size parameter for fairseq-interactive defined inside this joint_translate script


# here we are translating the english sentences to tamil
!bash joint_translate.sh en_sentences.txt ta_outputs.txt 'en' 'ta' '../indictrans-en-indic'

Mon Apr 12 14:35:44 UTC 2021
Applying normalization and script conversion
100% 3/3 [00:00<00:00, 58.78it/s]
Number of sentences in input: 3
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Mon Apr 12 14:37:59 UTC 2021
Translation completed


In [ ]:
# see the model outputs saved in ta_outputs.txt
!cat ta_outputs.txt

இந்த சைக்கிள் ரொம்ப சிறியது!
நான் தாமதமாக வருகிறேன். விமான நிலையத்தில் நேரடியாக உங்களை சந்திக்கிறேன்.
2006 ஆம் ஆண்டில், டைம் பத்திரிகை, எவரை எடிட் செய்ய அனுமதிக்கும் திறந்த கதவு கொள்கை, விக்கிப்பீடியாவை "உலகின் மிகப்பெரிய மற்றும் ஒருவேளை சிறந்த கலைக்களஞ்சியமாக" மாற்றியது, மேலும் ஜிம்மி வாலஸ்டின் தொலைநோக்குப் பார்வைக்கு ஒரு சான்றாக இருந்தது என்று கூறியது.


In [ ]:
# Similarly, we can translate the english sentences to hindi
!bash joint_translate.sh en_sentences.txt hi_outputs.txt 'en' 'hi' '../indictrans-en-indic'

Mon Apr 12 14:39:07 UTC 2021
Applying normalization and script conversion
100% 3/3 [00:00<00:00, 54.43it/s]
Number of sentences in input: 3
Applying BPE
Decoding
Extracting translations, script conversion and detokenization
Mon Apr 12 14:41:21 UTC 2021
Translation completed


In [ ]:
!cat hi_outputs.txt

यह साइकिल तुम्हारे लिए बहुत छोटी है!
मैं देर से आ रहा हूं, एयरपोर्ट पर आपसे सीधे मिलूंगा
2006 में, टाइम पत्रिका ने कहा कि किसी को भी संपादित करने की अनुमति देने की ओपन-डोर नीति ने विकिपीडिया को "दुनिया का सबसे बड़ा और शायद सबसे अच्छा विश्वकोश" बना दिया था, और जिमी वालेस्ट के दृष्टिकोण का एक वसीयतनामा बना दिया था।


In [ ]:
# to compute bleu scores for the predicitions with a reference file, use the following command

# bash compute_bleu.sh pred_fname ref_fname src_lang tgt_lang
# arguments:
# pred_fname: file that contains model predictions
# ref_fname: file that contains references
# src_lang and tgt_lang : the source and target language